In [1]:
import copy

In [2]:
class Path():
    def __init__(self):
        self.id = -1
        self.path = []
        self.new_id()
        self.double = ''
        
    def new_id(self):
        global path_counter
        self.id = path_counter + 1
        path_counter += 1
        
    def branch(self):
        newpath = copy.deepcopy(self)
        newpath.new_id()
        return newpath
    
    def append(self, cave):
        if cave.name.islower() and cave.name not in ('start', 'end'):
            if cave.name in [c.name for c in self.path] and self.double == '':
                # Small cave second visit
                #print(self)
                #print('Revisiting', cave.name)
                self.double = cave.name
        self.path.append(cave)
                
    def visited(self, cave):
        if cave.large:
            return False
        #print('Double', self.double)
        #print(self)
    
        if cave.name not in ('start', 'end'):
            if self.double == '':
                return False
            
        if cave.name in [c.name for c in self.path]:
            return True
        
        return False

    def __repr__(self):
        return repr(self.path)


In [3]:
class Cave():
    def __init__(self, name):
        self.large = name.isupper()
        self.name = name
        self._visited = []
        
        self.connections = []
        
    @property
    def visited(self):
        if self.large:
            return False
        return self._visited
    
    @property
    def is_end(self):
        return self.name == 'end'
        
    def connect2(self, dest):
        if dest not in self.connections:
            self.connections.append(dest)
        if self not in dest.connections:
            dest.connect2(self)
            
    def __repr__(self):
        return self.name
            
    
    def find_path(self, currpath, path_list, lvl=1):
        ''' Enter the cave and find the next exit'''
        # print(lvl,'Entering', self.name)
        currpath.append(self)
        
        if lvl>50:
            raise Exception
        
        if self.is_end:
            #print('Found end')
            return
        
        possible_connections = []
        for c in self.connections:
            if not currpath.visited(c):
                possible_connections.append(c)
        
        currbranch = currpath.branch()
                
        #print(lvl, possible_connections, 'for', currpath)

        for i, cave in enumerate(possible_connections):
            path = currpath
            #print(lvl, path.id,i,'Looking at', cave.name)

            if i > 0:
                #print('Branching')
                path = currbranch.branch()
                path.new_id()
                path_list.append(path)
                #print(path.path)
                cave.find_path(path, path_list, lvl+1)
            else:
                cave.find_path(currpath, path_list, lvl+1)

        return
            
    def search(self, target):
        currpath = [self.name]
        return target.find_path(currpath)


In [6]:
cave_map = {}
path_counter = 0

with open('input12_1.txt', 'r') as inf:
    for line in inf.readlines():
        c1, c2 = line.strip().split('-')
        print(c1, c2)
        if c1 not in cave_map:
            #print('Adding', c1)
            cave_map[c1] = Cave(c1)
        if c2 not in cave_map:
            #print('Adding', c2)
            cave_map[c2] = Cave(c2)
                
        # Add connections
        cave_map[c1].connect2(cave_map[c2])

path1 = Path()
path_list = [path1]

cave_map['start'].find_path(path1, path_list)

good_paths = []
for path in path_list:
    if 'end' in [c.name for c in path.path]:
        good_paths.append(path)
#print(good_paths)
print('N paths', len(good_paths))


start A
start b
A c
A b
b d
A end
b end
N paths 36
